# OptiGuide Example with Pyomo


Authors: [Phuc Van Phan](https://github.com/pphuc25), [Alfredo Hernandez](https://github.com/alfredoihernandez), [Beibin Li](https://github.com/beibinli)



For this notebook, we mainly introduce how to use Pyomo with OptiGuide, and we skipped the overview of OptiGuide what-if analysis. If you haven't checked the [OptiGuide example with Gurobi](optiguide_example.ipynb), please check it first.


In [1]:
# Install Required Packages
# %pip install optiguide

In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import autogen
import autogen
from autogen.agentchat import Agent, UserProxyAgent
from optiguide import OptiGuideAgent

In [3]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4o",
        }
    }
)

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [4]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/microsoft/OptiGuide/main/benchmark/application/coffee_pyomo.py"
response  = requests.get(code_url)


# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open("../benchmark/application/coffee_pyomo.py", "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

import time

from pyomo.environ import (ConcreteModel, Constraint, Integers, Objective,
                           SolverFactory, TerminationCondition, Var, minimize)

# Example data
capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}
shipping_cost_from_supplier_to_roastery = {
    ('supplier1', 'roastery1'): 5,
    ('supplier1', 'roastery2'): 4,
.
.
.

# You can change the solver as per your requirement
m = solver.solve(model)

print(time.ctime())
if m.solver.termination_condition == TerminationCondition.optimal:
    print(f'Optimal cost: {model.obj()}')
else:
    print("Not solved to optimality. Optimization status:",
          m.solver.termination_condition)



In [5]:
# In-context learning examples.
pyomo_example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
m.z = Var(suppliers, domain=Binary, bounds=(0, None))
def one_supplier_constraint(m):
    return sum(m.z[s] for s in suppliers) <= 1
m.OneSupplierConstraint = Constraint(rule=one_supplier_constraint)
def roastery_2_demand_met_constraint(m, s):
    return m.x[s,'roastery2'] <= capacity_in_supplier[s] * m.z[s]
m.Roastery2DemandMetConstraint = Constraint(suppliers, rule=roastery_2_demand_met_constraint)
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```
"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [6]:
agent = OptiGuideAgent(
    name="OptiGuideCoffeeExample",
    source_code=code,
    debug_times=5,
    example_qa=pyomo_example_qa,
    solver_software="pyomo",
    use_safeguard=True,
    llm_config={
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

Sun Oct 27 20:52:35 2024
Optimal cost: 2470.0


Now, let's create a user's question.

In [7]:
# user.initiate_chat(agent, message="What if we prohibit shipping from supplier 1 to roastery 2?")
ans = user.initiate_chat(agent, message="Why is it not recommended to use just one supplier for roastery 2?")

user (to OptiGuideCoffeeExample):

Why is it not recommended to use just one supplier for roastery 2?

--------------------------------------------------------------------------------
OptiGuideCoffeeExample (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuideCoffeeExample):

```python
m.z = Var(suppliers, domain=Binary, bounds=(0, None))

def one_supplier_constraint(m):
    return sum(m.z[s] for s in suppliers) <= 1

m.OneSupplierConstraint = Constraint(rule=one_supplier_constraint)

def roastery_2_demand_met_constraint(m, s):
    return m.x[s,'roastery2'] <= capacity_in_supplier[s] * m.z[s]

m.Roastery2DemandMetConstraint = Constraint(suppliers, rule=roastery_2_demand_met_constraint)
```

--------------------------------------------------------------------------------
OptiGuideCoffeeExample (to safeguard):


--- Code ---
m.z = Var(suppliers, domain=Binary, bounds=(0, None))

def one_supplier_constraint(m):


In [8]:
print(ans.summary)

The code was successfully updated to handle the scenario of using only one supplier for roastery 2. The updated constraints restrict the selection to just one supplier, which is reflected in the results.

Originally, the optimization problem provided an objective value of 2470.0. However, with the new restriction applied, the objective value increased to 2520.0. This indicates that enforcing the constraint of using just one supplier for roastery 2 raised the overall cost of the supply chain, demonstrating the trade-off between logistical flexibility and cost efficiency. 

Therefore, while restricting the supply to one supplier simplifies logistics, it may not be the most cost-effective solution.


In [9]:
ans2 = user.initiate_chat(agent, message="What is the impact of supplier1 being able to supply only half the quantity at present?")

user (to OptiGuideCoffeeExample):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
OptiGuideCoffeeExample (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuideCoffeeExample):

```python
capacity_in_supplier['supplier1'] = capacity_in_supplier['supplier1'] * 0.5
```

--------------------------------------------------------------------------------
OptiGuideCoffeeExample (to safeguard):


--- Code ---
capacity_in_supplier['supplier1'] = capacity_in_supplier['supplier1'] * 0.5

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuideCoffeeExample):

SAFE

--------------------------------------------------------------------------------
Sun Oct 27 20:52:42 2024
Not solved to optimality. Optimization status:

In [10]:
print(ans2.summary)

The impact of supplier1 being able to supply only half the quantity at present has led to the optimization problem becoming infeasible. This means that with the reduced supply capacity from supplier1, it is not possible to meet all demand requirements, including the supply to roasteries and eventually to the cafes.

Originally, the problem was solved optimally with an objective value of 2470.0, indicating a feasible solution that met all constraints such as supply, demand, and flow requirements. However, with the constraint on supplier1 halved, the supply from supplier1 is insufficient to meet the demands of the network while satisfying all constraints, resulting in an infeasible solution.

Therefore, adjustments in supply from other suppliers, alternative sourcing strategies, or demand management would be necessary to achieve feasibility under these new conditions.
